In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Spray Dataframe Exploratory Analysis

In [2]:
## Now i am importing the spray dataframe and printing out some characteristics
## of this dataframe

spray_df = pd.read_csv('/Users/TerryONeill/west_nile/west_nile/assets/spray.csv')
print 'size of dataframe:\n', spray_df.shape
print
print 'datatypes:\n', spray_df.dtypes
print
print 'null values:\n', spray_df.isnull().sum()  ## you can see we have some nulls here

size of dataframe:
(14835, 4)

datatypes:
Date          object
Time          object
Latitude     float64
Longitude    float64
dtype: object

null values:
Date           0
Time         584
Latitude       0
Longitude      0
dtype: int64


In [4]:
## now the head of this dataframe
spray_df.head()


,Date,Time,Latitude,Longitude
0,2011-08-29,6:56:58 PM,42.391623,-88.089163
1,2011-08-29,6:57:08 PM,42.391348,-88.089163
2,2011-08-29,6:57:18 PM,42.391022,-88.089157
3,2011-08-29,6:57:28 PM,42.390637,-88.089158
4,2011-08-29,6:57:38 PM,42.390410,-88.088858


In [3]:
## I am creating the engine that connects psql with my newly created database (west_nile)
## so that i can export the dataframes as tables

from sqlalchemy import create_engine
import psycopg2

engine = create_engine('postgresql://TerryONeill@localhost:5432/west_nile')

In [4]:
## this is adding the dataframe to my newly created database in psql as
## a table named 'whatever the first parameter is'
spray_df.to_sql('spray', engine)

## an error might get kicked up here since a table has already been created, that 
## is fine for us, just ignore the error. If you alter the dataframe then you need
## to delete the table in psql and then recreate it

ValueError: Table 'spray' already exists.